In [1]:
%%capture
!pip install pyspark

In [4]:
from sklearn.preprocessing import OneHotEncoder

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lab1-2').getOrCreate()

In [5]:
import numpy as np

In [42]:
df4 = spark.read.csv('/content/PatientInfo.csv', header=True)
df4.show(10)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

In [ ]:
#d=df4.select("Cruise_line").rdd.map(lambda x : x[0]).collect()

In [43]:
from pyspark.sql.functions import coalesce
df4=df4.withColumn("deceased_date",coalesce(df4.deceased_date,df4.released_date)) 

In [44]:
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

df4=df4.withColumn("deceased_date",col("deceased_date").cast(DateType())) \
.withColumn("confirmed_date",col("confirmed_date").cast(DateType()))

In [45]:
from pyspark.sql.functions import *
df4=df4.withColumn("no_days",datediff(col("deceased_date"),col("confirmed_date")))


In [46]:
df4=df4.withColumn("is_male",when( (col("sex") == "male"), 1).otherwise(0)) 

In [47]:
df4.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: integer (nullable = false)



In [48]:
df4=df4.withColumn("is_dead",when( (col("state") == "released"), 0).otherwise(1)) 

In [49]:
from pyspark.sql.functions import translate


df4=df4.withColumn('age', translate('age', 's', ''))

In [50]:
import pyspark.sql.types as t
df4=df4.withColumn("age",col("age").cast(t.DoubleType())) 

In [51]:
df4=df4.withColumn("no_days",col("no_days").cast(t.DoubleType()))

In [52]:
df4.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



In [53]:
df4=df4.drop("sex","confirmed_date","released_date","deceased_date","state")

In [54]:
df4.show(2)

+----------+----+-------+--------+-----------+---------------+-----------+--------------+------------------+-------+-------+-------+
|patient_id| age|country|province|       city| infection_case|infected_by|contact_number|symptom_onset_date|no_days|is_male|is_dead|
+----------+----+-------+--------+-----------+---------------+-----------+--------------+------------------+-------+-------+-------+
|1000000001|50.0|  Korea|   Seoul| Gangseo-gu|overseas inflow|       null|            75|        2020-01-22|   13.0|      1|      0|
|1000000002|30.0|  Korea|   Seoul|Jungnang-gu|overseas inflow|       null|            31|              null|   32.0|      1|      0|
+----------+----+-------+--------+-----------+---------------+-----------+--------------+------------------+-------+-------+-------+
only showing top 2 rows



In [56]:
df4=df4.drop("symptom_onset_date","contact_number","infected_by")

In [57]:
df4.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



In [58]:
enc = OneHotEncoder(handle_unknown='ignore')

In [59]:
d=df4.select("infection_case").rdd.map(lambda x : x[0]).collect()

In [60]:
A = np.array(list(d)).reshape(-1,1)

In [61]:
data_encoded=enc.fit_transform( A).toarray()
#data_encoded=data_encoded.tolist()

In [62]:
cols=enc.get_feature_names_out()
cols=cols.tolist()

In [63]:
import pandas as pd
df = pd.DataFrame(data_encoded,columns =cols)
df.head(10)

,x0_Anyang Gunpo Pastors Group,x0_Biblical Language study meeting,x0_Bonghwa Pureun Nursing Home,x0_Changnyeong Coin Karaoke,x0_Cheongdo Daenam Hospital,x0_Coupang Logistics Center,x0_Daejeon door-to-door sales,x0_Daezayeon Korea,x0_Day Care Center,x0_Dongan Church,...,x0_Yangcheon Table Tennis Club,x0_Yeonana News Class,x0_Yeongdeungpo Learning Institute,x0_Yongin Brothers,x0_contact with patient,x0_etc,x0_gym facility in Cheonan,x0_gym facility in Sejong,x0_overseas inflow,x0_None
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [64]:
deptDF = spark.createDataFrame(df)

In [65]:
deptDF.show(10)

+-----------------------------+----------------------------------+------------------------------+---------------------------+---------------------------+---------------------------+-----------------------------+------------------+------------------+----------------+--------------------------+--------------------------------+------------------------+---------------------------------+------------------+------------------------------------------------+----------------------------+----------------------+----------------------------------------+-----------------------------+--------------------------------+----------------+--------------------+---------------------------------+----------------+-----------------------------------+----------------+--------------+--------------+-----------------------+----------+----------------------------------+-----------------------------------+----------------------------------+-------------------------+----------------+-------------------+---------------

In [66]:
df4.printSchema()


root
 |-- patient_id: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



In [67]:
df4=df4.drop("infection_case") 

In [68]:
from pyspark.sql.functions import lit

In [69]:
for column in [column for column in deptDF.columns
               if column not in df4.columns]:
    df4 = df4.withColumn(column, lit(None))

In [70]:
from pyspark.sql.functions import monotonically_increasing_id 
df4 = df4.select("*").withColumn("id", monotonically_increasing_id())

In [71]:
deptDF=deptDF.select("*").withColumn("id", monotonically_increasing_id())

In [72]:
df4=df4.join(deptDF, df4.id ==deptDF.id).select(df4["*"],deptDF["*"])

In [73]:
df4=df4.drop("id")

In [74]:
df4.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)
 |-- x0_Anyang Gunpo Pastors Group: void (nullable = true)
 |-- x0_Biblical Language study meeting: void (nullable = true)
 |-- x0_Bonghwa Pureun Nursing Home: void (nullable = true)
 |-- x0_Changnyeong Coin Karaoke: void (nullable = true)
 |-- x0_Cheongdo Daenam Hospital: void (nullable = true)
 |-- x0_Coupang Logistics Center: void (nullable = true)
 |-- x0_Daejeon door-to-door sales: void (nullable = true)
 |-- x0_Daezayeon Korea: void (nullable = true)
 |-- x0_Day Care Center: void (nullable = true)
 |-- x0_Dongan Church: void (nullable = true)
 |-- x0_Dunsan Electronics Town: void (nullable = true)
 |-- x0_Eunpyeong St. Mary's Hospital: void (nullable = true)
 |-- x0_G

In [75]:
dd=df4.select("country").rdd.map(lambda x : x[0]).collect()

In [76]:
AA = np.array(list(dd)).reshape(-1,1)

In [77]:
data_encoded2=enc.fit_transform( AA).toarray()

In [78]:
colss=enc.get_feature_names_out()
colss=colss.tolist()

In [80]:
import pandas as pd
df2 = pd.DataFrame(data_encoded2,columns =colss)
df2.head(2)

,x0_Canada,x0_China,x0_Foreign,x0_France,x0_Germany,x0_India,x0_Indonesia,x0_Korea,x0_Mongolia,x0_Switzerland,x0_Thailand,x0_United States,x0_Vietnam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [81]:
deptDF2 = spark.createDataFrame(df2)

In [82]:
df4=df4.drop("country")

In [83]:
for column in [column for column in deptDF2.columns
               if column not in df4.columns]:
    df4 = df4.withColumn(column, lit(None))

In [84]:
df4 = df4.select("*").withColumn("id", monotonically_increasing_id())

In [85]:
deptDF2=deptDF2.select("*").withColumn("id", monotonically_increasing_id())

In [86]:
df4=df4.join(deptDF2, df4.id ==deptDF2.id).select(df4["*"],deptDF2["*"])

In [87]:
df4=df4.drop("id")

In [89]:
df4=df4.drop("patient_id","province","city")

In [90]:
df4.printSchema()

root
 |-- age: double (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)
 |-- x0_Anyang Gunpo Pastors Group: void (nullable = true)
 |-- x0_Biblical Language study meeting: void (nullable = true)
 |-- x0_Bonghwa Pureun Nursing Home: void (nullable = true)
 |-- x0_Changnyeong Coin Karaoke: void (nullable = true)
 |-- x0_Cheongdo Daenam Hospital: void (nullable = true)
 |-- x0_Coupang Logistics Center: void (nullable = true)
 |-- x0_Daejeon door-to-door sales: void (nullable = true)
 |-- x0_Daezayeon Korea: void (nullable = true)
 |-- x0_Day Care Center: void (nullable = true)
 |-- x0_Dongan Church: void (nullable = true)
 |-- x0_Dunsan Electronics Town: void (nullable = true)
 |-- x0_Eunpyeong St. Mary's Hospital: void (nullable = true)
 |-- x0_Gangnam Dongin Church: void (nullable = true)
 |-- x0_Gangnam Yeoksam-dong gathering: void (nullable = true)
 |-- x0_Geochang Church: void (nullable = true)
 

In [ ]:
age: double (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: in

#Imputer

#Use Imputer to fill missing data with mean.

In [92]:
 from pyspark.ml.feature import Imputer

In [93]:
imputer = Imputer()

In [94]:
imputer.setInputCols(["age", "no_days","is_male","is_dead"])

Imputer_d396e58c6d3c

In [95]:
imputer.setOutputCols(["age_", "no_days_","is_male_","is_dead_"])

Imputer_d396e58c6d3c

In [99]:
df=imputer.setStrategy("median").setMissingValue(1.0).fit(df4).transform(df4)

In [102]:
df=df.drop("age", "no_days","is_male","is_dead")

In [103]:
df.printSchema()

root
 |-- x0_Anyang Gunpo Pastors Group: void (nullable = true)
 |-- x0_Biblical Language study meeting: void (nullable = true)
 |-- x0_Bonghwa Pureun Nursing Home: void (nullable = true)
 |-- x0_Changnyeong Coin Karaoke: void (nullable = true)
 |-- x0_Cheongdo Daenam Hospital: void (nullable = true)
 |-- x0_Coupang Logistics Center: void (nullable = true)
 |-- x0_Daejeon door-to-door sales: void (nullable = true)
 |-- x0_Daezayeon Korea: void (nullable = true)
 |-- x0_Day Care Center: void (nullable = true)
 |-- x0_Dongan Church: void (nullable = true)
 |-- x0_Dunsan Electronics Town: void (nullable = true)
 |-- x0_Eunpyeong St. Mary's Hospital: void (nullable = true)
 |-- x0_Gangnam Dongin Church: void (nullable = true)
 |-- x0_Gangnam Yeoksam-dong gathering: void (nullable = true)
 |-- x0_Geochang Church: void (nullable = true)
 |-- x0_Geumcheon-gu rice milling machine manufacture: void (nullable = true)
 |-- x0_Guri Collective Infection: void (nullable = true)
 |-- x0_Guro-gu Call 

In [104]:
from pyspark.ml.feature import VectorAssembler

In [107]:
df.select(colss).show(2)

AnalysisException: ignored

In [105]:
s=df.schema

In [111]:
input_f=["age_", "no_days_","is_male_","is_dead_"]+colss

In [160]:
input_f=["age_", "no_days_","is_male_","is_dead_"]

In [161]:
vecAssembler = VectorAssembler(inputCols=input_f,outputCol='features')

In [113]:
from sklearn.model_selection import train_test_split


In [ ]:
#x=df4.select(input_f)

In [ ]:
#x.show(20)

In [114]:
y=df.select("is_dead_")

In [166]:
train_df,test_df=train_test_split(df.toPandas(), test_size=0.10, random_state=42)

In [116]:
from pyspark.sql.types import StructType, StructField, StringType,IntegerType,FloatType

In [117]:
from pyspark.sql.types import *

In [119]:
s_y=StructType([StructField('is_dead_', IntegerType(), True)])

In [167]:
train_xx=spark.createDataFrame(train_df[input_f])#schema=s_x

'''
train_y=spark.createDataFrame(train_target)
test_y=spark.createDataFrame(test_target)
'''

'\ntrain_y=spark.createDataFrame(train_target)\ntest_y=spark.createDataFrame(test_target)\n'

In [165]:
input_f

['age_', 'no_days_', 'is_male_', 'is_dead_']

In [168]:
test_xx=spark.createDataFrame(test_df[input_f])

In [169]:
train_x

DataFrame[age_: double, no_days_: double, is_male_: bigint, features: vector]

In [170]:
train_x = vecAssembler.transform(train_xx)

In [171]:
test_x = vecAssembler.transform(test_xx)

In [172]:
test_x

DataFrame[age_: double, no_days_: double, is_male_: bigint, is_dead_: bigint, features: vector]

In [186]:
#Create a DecisionTreeClassifier Model

In [185]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [188]:
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="is_dead_")
dtc = dtc.fit(train_x)

pred = dtc.transform(test_x)
pred.show(3)
 

+----+--------+--------+--------+-------------------+-------------+-----------+----------+
|age_|no_days_|is_male_|is_dead_|           features|rawPrediction|probability|prediction|
+----+--------+--------+--------+-------------------+-------------+-----------+----------+
|20.0|    16.0|       0|       0|[20.0,16.0,0.0,0.0]|      [921.0]|      [1.0]|       0.0|
|20.0|    16.0|       0|       0|[20.0,16.0,0.0,0.0]|      [921.0]|      [1.0]|       0.0|
|20.0|    16.0|       0|       0|[20.0,16.0,0.0,0.0]|      [921.0]|      [1.0]|       0.0|
+----+--------+--------+--------+-------------------+-------------+-----------+----------+
only showing top 3 rows



In [194]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol='is_dead_')
acc = evaluator.evaluate(pred)

In [196]:
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  1.0


In [197]:
#Creating a Pipeline

In [198]:
from pyspark.ml import Pipeline

In [199]:
pipeline = Pipeline(stages=[vecAssembler,dtc])

In [205]:
input_f

['age_', 'no_days_', 'is_male_', 'is_dead_']

In [213]:
df44 = df.withColumnRenamed("age","age_")

In [206]:
pipelineModel = pipeline.fit(df44[["age", "no_days","is_male","is_dead"]])

In [216]:
dd=df44.select(["age_", "no_days_", "is_dead_", "is_male_"]).limit(20)

In [217]:
predDF = pipelineModel.transform(dd)

In [218]:
predDF.select(["features",'is_dead_','prediction']).show()

+-------------------+--------+----------+
|           features|is_dead_|prediction|
+-------------------+--------+----------+
|[50.0,13.0,0.0,0.0]|       0|       0.0|
|[30.0,32.0,0.0,0.0]|       0|       0.0|
|[50.0,20.0,0.0,0.0]|       0|       0.0|
|[20.0,16.0,0.0,0.0]|       0|       0.0|
|[20.0,24.0,0.0,0.0]|       0|       0.0|
|[50.0,19.0,0.0,0.0]|       0|       0.0|
|[20.0,10.0,0.0,0.0]|       0|       0.0|
|[20.0,22.0,0.0,0.0]|       0|       0.0|
|[30.0,16.0,0.0,0.0]|       0|       0.0|
|[60.0,24.0,0.0,0.0]|       0|       0.0|
|[50.0,23.0,0.0,0.0]|       0|       0.0|
|[20.0,20.0,0.0,0.0]|       0|       0.0|
|[80.0,16.0,0.0,0.0]|       0|       0.0|
|[60.0,25.0,0.0,0.0]|       0|       0.0|
|[70.0,16.0,0.0,0.0]|       0|       0.0|
|[70.0,21.0,0.0,0.0]|       0|       0.0|
|[70.0,10.0,0.0,0.0]|       0|       0.0|
|[20.0,16.0,0.0,0.0]|       0|       0.0|
|[70.0,17.0,0.0,0.0]|       0|       0.0|
|[70.0,16.0,0.0,0.0]|       0|       0.0|
+-------------------+--------+----